# Notebook Instructions

1. If you are new to Jupyter notebooks, please go through this introductory manual <a href='https://quantra.quantinsti.com/quantra-notebook' target="_blank">here</a>.
1. Any changes made in this notebook would be lost after you close the browser window. **You can download the notebook to save your work on your PC.**
1. Before running this notebook on your local PC:<br>
i.  You need to set up a Python environment and the relevant packages on your local PC. To do so, go through the section on "**Run Codes Locally on Your Machine**" in the course.<br>
ii. You need to **download the zip file available in the last unit** of this course. The zip file contains the data files and/or python modules that might be required to run this notebook.

# Implied Volatility (IV) 

The implied volatility represents the volatility in the underlying security that is expected by the market participants. It is also a very useful measure for forecasting the underlying security's future price movements.

In this notebook, we will understand how to calculate implied volatility (IV). 

#### The notebook is structured as follows: 

1. [Import Libraries](#libraries)
2. [Import the Data](#data)
3. [Calculate Implied Volatility (IV)](#volatility)

<a id='libraries'></a>
## Import Libraries

In [1]:
# For data manipulation
import pandas as pd

# To calculate implied volatility
import mibian

# For datetime manipulation
import datetime

# To ignore warnings
import warnings
warnings.filterwarnings("ignore")

<a id='data'></a>
## Import the Data

The S&P 500 futures and options data is stored in a `spx_eom_options_2022.csv` file. This data covers the period from 03-01-2022 to 30-09-2022. You can download this data from the last unit of this course '**Python Codes and Data**'.

In [2]:
# Read the data
data = pd.read_csv("../data_modules/spx_eom_options_2022.csv")

# Convert to datetime
data['[QUOTE_DATE]'] = pd.to_datetime(data['[QUOTE_DATE]'], format='%d-%m-%Y')
data['[EXPIRE_DATE]'] = pd.to_datetime(data['[EXPIRE_DATE]'], format='%Y-%m-%d')

# Set the index
data = data.set_index(['[QUOTE_DATE]'])

# Display the top 5 rows
data.head()

,[STRIKE],[C_LAST],[UNDERLYING_LAST],[P_LAST],[EXPIRE_DATE],[RATE],[DTE]
[QUOTE_DATE],,,,,,,
2022-01-03,800,0.0,4795.57,0.15,2022-01-31,0.4,28
2022-01-03,1000,0.0,4795.57,0.22,2022-01-31,0.4,28
2022-01-03,1200,0.0,4795.57,0.25,2022-01-31,0.4,28
2022-01-03,1300,0.0,4795.57,0.05,2022-01-31,0.4,28
2022-01-03,1400,0.0,4795.57,0.25,2022-01-31,0.4,28


<a id='volatility'></a>
## Calculate Implied Volatility (IV)

Implied volatility is a prediction of movement in an options contract's price by the market participants. It represents the expected fluctuations in the price of the contract till the expiration. 

To calculate the IV, we can use the `impliedVolatility` function of the BS (Black Scholes) class in the `mibian` library. You can check out the documentation of the `mibian` library <a href='https://github.com/yassinemaaroufi/MibianLib/blob/master/README' target="_blank">here</a>. 

The syntax for the `mibian` Black Scholes function is:

```python
mibian.BS()
```
Parameters: 

* Close price of the option
* Strike price of the option
* Interest rate
* Time to expiry (If the days to expiry is 0 or the underlying price is 0, then the value of IV will be 0.)
* Price of the call or put option
        
Let's define a function to calculate the IV values. 

In [3]:
# Function to calculate the IV
def calculate_IV(data, option_type):
    
    # Return IV equals 0 if days to expiry is 0 or the last traded price is 0 
    if ((data['[UNDERLYING_LAST]'][0] == 0) or (data['[DTE]'][0] == 0)):
        return 0

    # Calculate the IV for option
    elif (option_type == 'CE'):
        IV = mibian.BS([data['[UNDERLYING_LAST]'][0], data['[STRIKE]'][0], data['[RATE]'][0],
                          data['[DTE]'][0]], callPrice=data['[C_LAST]'][0]).impliedVolatility
        return IV
    
    elif (option_type == 'PE'):
        IV =  mibian.BS([data['[UNDERLYING_LAST]'][0], data['[STRIKE]'][0], data['[RATE]'][0],
                          data['[DTE]'][0]], putPrice=data['[P_LAST]'][0]).impliedVolatility
        return IV
        
    

In the above cell, we have defined a function to calculate the IV. Now, we will calculate the IV value for the strike price of 4795 on 3rd January 2022. 

<b> Note: </b> The following cell may take some time to run.

In [4]:
# Calculate the IV
iv = calculate_IV(data[(data.index == datetime.datetime(2022, 1, 3)) & (data['[STRIKE]'] == 4795)], "CE")

# Display the IV
print("IV =", iv)

IV = 11.363983154296875


The IV on 3rd January 2022 for the 4795 strike call option is approximately 11.36. This means that the underlying security is expected to move approximately 11.36% annually.

## Conclusion

In this notebook, we calculated the implied volatility of an options contract. Similarly, you can try calculating IVs of different options contracts. In the upcoming section, we will see how we can forecast implied volatility using machine learning.<br><br>